In [1]:
import os
from dotenv import load_dotenv
from image_helper import ImageHelper
from datetime import datetime

load_dotenv()

base_url = os.environ.get("STORAGE_URL")
# base_url will be empty if .env file is missing
assert base_url != ""
im_helper = ImageHelper(base_url)

In [3]:
dt_str = '25/11/2022 17:08:18' 
dt = im_helper.dt_from_string(dt_str)
if im_helper.get(dt, "top"):
    im_helper.show()